In [13]:
# pip install -U langchain-community langchain-huggingface pypdf sentence-transformers

In [14]:
print("ok")

ok


In [15]:
%pwd            #print working directory

'd:\\Python\\Python Project\\Langchain-Project\\Medical-Chatbot-GenAI\\research'

In [29]:
import os
os.chdir("../")

In [17]:
# %pwd

In [18]:
# import os
# os.chdir("Medical-Chatbot-GenAI")

In [30]:
%pwd

'd:\\Python\\Python Project\\Langchain-Project\\Medical-Chatbot-GenAI'

In [31]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls = PyPDFLoader)
    
    documents = loader.load()
    return documents
##

In [ ]:
extracted_data = load_pdf_file(data = "Data/")          ##

In [ ]:
extracted_data

In [ ]:
# Split the data into text chunks
def text_split(extracted_data):                 ##
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks
#done

In [43]:
# Split the extracted data into text chunks and print how many chunks were created
text_chunks = text_split(extracted_data)        ##
print("Length of text Chunks", len(text_chunks))

Length of text Chunks 5859


In [ ]:
text_chunks

In [ ]:
# Import HuggingFaceEmbeddings to convert text into vector(numerical) embeddings using Hugging Face models

from langchain_huggingface import HuggingFaceEmbeddings     ##

In [ ]:
#Downnnnnnload the Embedings from Hugging Face

def  download_hugging_face_embeddings():        ##
    embeddings =  HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings() ##

In [36]:
query_result = embeddings.embed_query("Hello World")
print("length", len(query_result))

length 384


In [37]:
# query_result

In [38]:
# pip install pinecone

In [39]:
from dotenv import load_dotenv      ##
import os

load_dotenv(override=True)
# print(os.getenv("PINECONE_API_KEY"))

True

In [57]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GOOGLE_API_KEY=os.environ.get('GOOGLE_API_KEY')

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone 
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)
##

In [58]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [ ]:
# pip install -U langchain-pinecone

In [ ]:
# Convert text chunks into embeddings and store them in Pinecone for semantic search

from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)
##

In [47]:
# load Existing Index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [46]:
docsearch

In [48]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [49]:
retrieved_docs = retriver.invoke("What is acne?")

In [50]:
retrieved_docs

[Document(id='9eb36323-4718-436b-87cf-5cf5d217e56d', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='eda3c252-cfa1-4b3e-a47d-30b29dd10df3', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='aaa1662f-8bbe-428b-901c-8cbe1bde4ae4', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'Py

In [ ]:
# pip install langchain-google-genai

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
# from dotenv import load_dotenv                        #I already initialize dotenv file for gemini api key , that's why no need to define multiple time

# load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.4,
    max_output_tokens=500
)

In [60]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [61]:
question_ans_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,question_ans_chain)

In [62]:
response = rag_chain.invoke({"input":"what is acne?"})
print(response["answer"])

Acne is a skin disorder where the sebaceous glands become inflamed.  It's also known as acne vulgaris.  The provided text includes a photograph illustrating acne on a woman's face.


In [64]:
response = rag_chain.invoke({"input":"what is fever?"})
print(response["answer"])

Based on the provided text, fever is a symptom of adenovirus infection in children,  characterized by an elevated body temperature (38°–40°C).  Additional symptoms may include general malaise, muscle aches, headache, chills, and loss of appetite.
